# Coursera Capstone - Toronto Project

## Part 1

In [1]:
import pandas as pd

### Import Table from Wikipedia into Dataframe

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
nhoods = pd.read_html(link)[0]

### Remove rows with 'Not assigned' Boroughs, replace 'Not assigned' neighbourhoods, and group by postcode

In [3]:
nhoods = nhoods[nhoods.Borough != 'Not assigned']
nhoods.Neighbourhood.replace("Not assigned",nhoods.Borough, inplace=True)
nhoods = nhoods.groupby('Postcode', as_index=False).agg({'Borough':'first','Neighbourhood':lambda x:", ".join(x)})
nhoods.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [4]:
nhoods.shape

(103, 3)

## Part 2

In [5]:
geo = pd.read_csv('https://cocl.us/Geospatial_data') # Read geo csv file because geocoder library didn't work

In [6]:
nhoods = nhoods.join(geo.set_index('Postal Code'), on='Postcode') # add latitude and longitude to neighbourhood data frame, matched on post code
nhoods.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Part 3

### Display map of Toronto with markers for each neighbourhood

In [8]:
!pip install folium
import folium

map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# add markers to map
for lat, lng, label in zip(nhoods['Latitude'], nhoods['Longitude'], nhoods['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

     |████████████████████████████████| 92kB 16.3MB/s eta 0:00:01


## Get venue data from FourSquare

In [9]:
CLIENT_ID = '0T1HYTMBMSMFT4D4VY1S4G3ID0XYRFXUK1NLVVZDC3DHITW1' # your Foursquare ID
CLIENT_SECRET = 'FZ0TMH0ZPLP5BGTI3FMG0GZF05PMCHPLUHKUQUJ3E4XEMLPH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [23]:
import requests

def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
toronto_venues = getNearbyVenues(names=nhoods['Neighbourhood'],
                                   latitudes=nhoods['Latitude'],
                                   longitudes=nhoods['Longitude']
                                  )

In [25]:
print(toronto_venues.shape)
toronto_venues.head()

(2241, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target


## Analyse Neighbourhoods

In [28]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

There are 272 uniques categories.


In [30]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

(101, 272)

## Cluster Neighbourhoods

In [33]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)

array([4, 4, 2, 1, 1, 1, 4, 4, 4, 4], dtype=int32)

In [69]:
# attach clu

#toronto_grouped.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = nhoods.join(toronto_grouped.set_index('Neighborhood'), on='Neighbourhood')

In [68]:
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import math
# create map
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if math.isnan(cluster):
        continue
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[(int)(cluster)-1],
        fill=True,
        fill_color=rainbow[(int)(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters